<a href="https://colab.research.google.com/github/JUBRIL-ADEBOGUN/Crop-yield-modelling/blob/main/Crop_yield_expeimentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing required libraries.
import pandas as pd, seaborn as sns
import numpy as np, random
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder

In [ ]:
# Set seed for reproducability

SEED = 2023
random.seed(SEED)
np.random.seed(SEED)
pd.set_option('display.max_columns', None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load files
data_path = '/content/drive/MyDrive/Digital Ocean /'
train = pd.read_csv(data_path + 'cleaned_train.csv')

In [ ]:
train.head()

,CultLand,CropCultLand,CropTillageDepth,SeedlingsPerPit,TransplantingIrrigationHours,TransIrriCost,StandingWater,Ganaura,CropOrgFYM,NoFertilizerAppln,BasalDAP,BasalUrea,1tdUrea,1appDaysUrea,2tdUrea,2appDaysUrea,Harv_hand_rent,Residue_length,Residue_perc,Acre,Yield,ID,District,Block,LandPreparationMethod,CropEstMethod,NursDetFactor,TransDetFactor,TransplantingIrrigationSource,TransplantingIrrigationPowerSource,OrgFertilizers,PCropSolidOrgFertAppMethod,CropbasalFerts,MineralFertAppMethod,FirstTopDressFert,MineralFertAppMethod.1,Harv_method,Threshing_method,Stubble_use,Threshing_date,Harv_date,SeedingSowingTransplanting,RcNursEstDate,CropTillageDate
0,45,40,5,2,5,200.0,2,0,0,2,0,20,15,18,0,0,0.0,30,40,0.312500,600,ID_GTFAC7PEVWQ9,Nalanda,Noorsarai,TractorPlough FourWheelTracRotavator,Manual_PuddledRandom,CalendarDate IrrigWaterAvailability SeedAvaila...,CalendarDate SeedlingAge RainArrival IrrigWate...,Boring,Electric,None,None,Urea,Broadcasting,Urea,Broadcasting,machine,machine,plowed_in_soil,2022-11-16,2022-11-16,2022-07-21,2022-06-27,2022-07-20
1,26,26,5,2,5,125.0,3,0,0,2,15,10,20,39,0,0,3.0,24,10,0.312500,600,ID_TK40ARLSPOKS,Nalanda,Rajgir,WetTillagePuddling TractorPlough FourWheelTrac...,Manual_PuddledRandom,CalendarDate PreMonsoonShowers IrrigWaterAvail...,CalendarDate SeedlingAge RainArrival IrrigWate...,Boring,Electric,None,None,DAP Urea,Broadcasting,Urea,Broadcasting,hand,machine,plowed_in_soil,2022-12-24,2022-11-25,2022-07-20,2022-06-20,2022-07-18
2,10,10,6,2,4,80.0,2,1,1,2,4,0,5,65,0,0,480.0,30,10,0.148148,225,ID_1FJY2CRIMLZZ,Gaya,Gurua,TractorPlough FourWheelTracRotavator,Manual_PuddledRandom,PreMonsoonShowers IrrigWaterAvailability Labou...,SeedlingAge IrrigWaterAvailability LaborAvaila...,Boring,Electric,Ganaura FYM,SoilApplied,DAP,SoilApplied,Urea,RootApplication,hand,machine,plowed_in_soil,2023-01-11,2022-12-12,2022-08-13,2022-06-20,2022-06-30
3,15,15,6,2,0,0.0,0,1,0,2,6,3,5,5,0,0,240.0,26,10,0.222222,468,ID_I3IPXS4DB7NE,Gaya,Gurua,TractorPlough FourWheelTracRotavator,Manual_PuddledRandom,CalendarDate PreMonsoonShowers IrrigWaterAvail...,CalendarDate SeedlingAge RainArrival IrrigWate...,None,None,Ganaura,SoilApplied,DAP Urea,Broadcasting,Urea,Broadcasting,hand,hand,plowed_in_soil,2022-12-29,2022-12-02,2022-07-17,2022-06-17,2022-06-16
4,60,60,4,2,9,300.0,2,0,0,2,15,30,30,26,0,0,0.0,24,40,0.468750,550,ID_4T8YQWXWHB4A,Nalanda,Noorsarai,other,Manual_PuddledRandom,CalendarDate IrrigWaterAvailability SeedAvaila...,SeedlingAge RainArrival IrrigWaterAvailability...,Boring,Electric,None,None,Urea DAP,Broadcasting,Urea,Broadcasting,machine,machine,plowed_in_soil,2022-12-02,2022-11-30,2022-07-20,2022-06-21,2022-07-19


# PREPROCESSING DATA.

In [ ]:
def preprocess(df, norm=False):
    data = df.drop('ID', axis=1)

    print('Data type sorting in progress......')
    # sort columns by data types.
    #dates = data.select_dtypes('date')
    nums = data.select_dtypes(exclude='object')
    cats = data.select_dtypes('object').columns

    # sort categorical data by cardinality.
    low_cad = [col for col in cats if data[col].nunique() < 10]
    high_cad = [col for col in cats if data[col].nunique() < 11]

    print('converting low cardinal columns to binary.........')
    # onehot encode low cardinal data
    encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype=np.int64)
    encoder.fit(data[low_cad])
    lowcad_df = pd.DataFrame(encoder.transform(data[low_cad]), dtype=np.int64)
    lowcad_df.columns = encoder.get_feature_names_out(low_cad)

    #print('converting high cardinal columns into numbers........')
    # ordinal encode high cardinal data.
    ordina= OrdinalEncoder()

    highcad_df = pd.DataFrame(ordina.fit_transform(data[high_cad]),dtype=np.int64)
    highcad_df.columns= high_cad

    #concat the dataframes together.
    print(' concatenating the data frames...')
    df = pd.concat([nums, highcad_df], axis=1)

    if norm is True:

        print('Normalizing dataframe....')
        # Normalize data.
        scaler = RobustScaler()

        df1 = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
        df = df1
    print('Done')

    return df

In [ ]:
df= train.copy()

df1 = preprocess(df, norm=False)

Data type sorting in progress......
converting low cardinal columns to binary.........
 concatenating the data frames...
Done


In [ ]:
df1.head()

,CultLand,CropCultLand,CropTillageDepth,SeedlingsPerPit,TransplantingIrrigationHours,TransIrriCost,StandingWater,Ganaura,CropOrgFYM,NoFertilizerAppln,BasalDAP,BasalUrea,1tdUrea,1appDaysUrea,2tdUrea,2appDaysUrea,Harv_hand_rent,Residue_length,Residue_perc,Acre,Yield,District,Block,LandPreparationMethod,CropEstMethod,TransplantingIrrigationSource,TransplantingIrrigationPowerSource,OrgFertilizers,PCropSolidOrgFertAppMethod,CropbasalFerts,MineralFertAppMethod,FirstTopDressFert,MineralFertAppMethod.1,Harv_method,Threshing_method,Stubble_use
0,45,40,5,2,5,200.0,2,0,0,2,0,20,15,18,0,0,0.0,30,40,0.312500,600,2,6,3,3,0,1,3,1,3,0,2,0,1,1,1
1,26,26,5,2,5,125.0,3,0,0,2,15,10,20,39,0,0,3.0,24,10,0.312500,600,2,7,7,3,0,1,3,1,1,0,2,0,0,1,1
2,10,10,6,2,4,80.0,2,1,1,2,4,0,5,65,0,0,480.0,30,10,0.148148,225,0,2,3,3,0,1,2,3,0,2,2,2,0,1,1
3,15,15,6,2,0,0.0,0,1,0,2,6,3,5,5,0,0,240.0,26,10,0.222222,468,0,2,3,3,2,2,1,3,1,0,2,0,0,0,1
4,60,60,4,2,9,300.0,2,0,0,2,15,30,30,26,0,0,0.0,24,40,0.468750,550,2,6,8,3,0,1,3,1,4,0,2,0,1,1,1


# MODELLING.

In [ ]:

from sklearn import model_selection


def create_folds(data, target:str):
    # we create a new column called kfold and fill it with -1
    data.loc[:,"kfold"] = -1
    # the next step is to randomize the rows of the data
    data = data.sample(frac=1).reset_index(drop=True)
    # calculate the number of bins by Sturge's rule       # I take the floor of the value, you can also
    # just round it
    num_bins = int(np.floor(1 + np.log2(len(data))))     # bin targets
    data.loc[:, "bins"] = pd.cut(data[target], bins=num_bins, labels=False)
    # initiate the kfold class from model_selection module
    kf = model_selection.StratifiedKFold(n_splits=5)
    # fill the new kfold column
    # data.bins.values
    # note that, instead of targets, we use bins!
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data.Block)):
        data.loc[v_, 'kfold'] = f
        # drop the bins column
        #data = data.drop("bins", axis=1)
        # return dataframe with folds
    return data

In [ ]:
df_fold = create_folds(df1, target='Yield')

In [ ]:
#model_dispatcher.py
from sklearn import ensemble
from sklearn import tree
from xgboost import XGBRegressor as xgbmodel
from  lightgbm import LGBMRegressor as lgbmodel

models = { "lgb": lgbmodel(n_estimators=950,random_state=20,),
           "xgb": xgbmodel(n_estimators=100,seed=23),
            "rf": ensemble.RandomForestClassifier(n_estimators=100,random_state = 123),}

In [ ]:
from sklearn import metrics
from sklearn import tree
#from utilities import create_folds

cols = ['CultLand', 'CropCultLand', 'TransIrriCost', 'Harv_hand_rent', 'cluster']

def run(fold, model):
    # read the training data with folds
    #df = pd.read_csv(config.TRAINING_FILE)
    df= df_fold.copy()
    #df['Yield'] = Y_train
    #tests= test_df.copy()
    #valid['Yield'] = Y_test
    # training data is where kfold is not equal to provided fold
    #df = create_folds(df, target='Acre')
    # also, note that we reset the index
    df_train = df[df.kfold != fold].reset_index(drop=True)
    # validation data is where kfold is equal to provided fold
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    # drop the label column from dataframe and convert it to # a numpy array by using .values.
    # target is label column in the dataframe
    x_train = df_train.drop(['kfold','Yield'], axis=1)
    y_train = df_train['Yield'].values
    # similarly, for validation, we have
    x_valid = df_valid.drop(['kfold', 'Yield'], axis=1)
    y_valid = df_valid['Yield'].values

    #print(x_train.shape, tests.shape)
   # fetch the model from model_dispatcher
    clf = models[model]
    # fir the model on training data
    clf.fit(x_train.drop(cols, axis=1), y_train)
    # create predictions for validation samples
    preds = clf.predict(x_valid.drop(cols, axis=1))
    # calculate & print accuracy
    accuracy = metrics.mean_squared_error(y_valid, preds, squared=False)
    print(f"Fold={fold}, MSE = {accuracy}")
    # save the model
    #joblib.dump( clf,  os.path.join(config.MODEL_OUTPUT, f"dt_{fold}.bin") )
    #test_preds = clf.predict(test_df.drop('Yield', axis=1))
    #accuracy = metrics.mean_squared_error(Y_test, valids, squared=False)
    #print(f"Valid Fold={fold}, Error = {accuracy}")
    #return test_preds

if __name__ == "__main__":
     mod = 'xgb'
     run(fold=0, model=mod)
     run(fold=1, model=mod)
     run(fold=2, model=mod)
     run(fold=3, model=mod)
     run(fold=4, model=mod)

Fold=0, Error = 55.7316515236636
Fold=1, Error = 56.494308452112456
Fold=2, Error = 55.464026564059196
Fold=3, Error = 59.400872776279996
Fold=4, Error = 58.26215972394347
